In [1]:
import numpy as np
import os
import scipy.linalg as la
from typing import Dict, List, Tuple, Optional  # Fix for NameError 
import rmsynth
from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import dump
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import Diagonal, StatePreparation

In [2]:
def extract_a_coeffs_from_diagonal_circuit(qc):
    """
    Extract Z8 phase coefficients from a *diagonal* unitary.
    Valid ONLY if qc is diagonal in computational basis.
    """
    U = Operator(qc).data
    diag = np.diag(U)

    a_coeffs = {}
    for i, val in enumerate(diag):
        phase = np.angle(val)
        if abs(phase) > 1e-12:
            k = int(round(4 * phase / np.pi)) % 8
            if k != 0:
                a_coeffs[i] = k

    return a_coeffs


In [3]:
def save_and_optimize_all(qc, name, n_qubits):
    """
    Uses rmsynth ONLY for diagonal / phase-polynomial circuits.
    Falls back to Qiskit otherwise.
    """

    try:
        if name != "11_diagonal":
            raise ValueError("rmsynth only valid for diagonal circuits")

        # ---- coefficient extraction ----
        phi_vals = [
            0, np.pi, 1.25*np.pi, 1.75*np.pi,
            1.25*np.pi, 1.75*np.pi, 1.5*np.pi, 1.5*np.pi,
            1.25*np.pi, 1.75*np.pi, 1.5*np.pi, 1.5*np.pi,
            1.5*np.pi, 1.5*np.pi, 1.75*np.pi, 1.25*np.pi
        ]

        a_coeffs = {
            i: int(round(4 * p / np.pi)) % 8
            for i, p in enumerate(phi_vals)
            if abs(p) > 1e-12
        }

        # ---- optimize phase polynomial ----
        vec_opt, report = rmsynth.optimize_coefficients(
            a_coeffs, n_qubits
        )

        circ = rmsynth.synthesize_from_coeffs(vec_opt, n_qubits)

        # ---- write OpenQASM ----
        with open(f"{name}.qasm", "w") as f:
            f.write("OPENQASM 2.0;\n")
            f.write('include "qelib1.inc";\n')
            f.write(f"qreg q[{n_qubits}];\n")

            for op in circ.ops:
                if op.kind == "cnot":
                    f.write(f"cx q[{op.ctrl}], q[{op.tgt}];\n")
                elif op.kind == "phase":
                    if op.k == 1:
                        f.write(f"t q[{op.q}];\n")
                    elif op.k == 7:
                        f.write(f"tdg q[{op.q}];\n")
                    elif op.k == 4:
                        # Z = T^4
                        for _ in range(4):
                            f.write(f"t q[{op.q}];\n")

        print(f"✨ {name:18s} | Optimized T-count ≈ {circ.t_count()}")

        return circ

    except Exception as e:
        print(f"⚠️ rmsynth skipped for {name}: {e}")

        qc_fallback = transpile(
            qc,
            basis_gates=["h", "t", "tdg", "cx"],
            optimization_level=3
        )
        dump(qc_fallback, f"{name}.qasm")
        return qc_fallback
